In [75]:
import psycopg2
import pandas as pd
import psycopg2.extras as extras

In [107]:
client = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/client.csv",sep=';')
date = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/date.csv",sep=';')
hotel = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/hotel.csv",sep=';')
room = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/room.csv",sep=';')
reservation = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/reservation.csv",sep=';')


# Oracle

In [88]:
#??

# MySQL

In [89]:
#??

# PostgreSQL

In [90]:
#https://waytolearnx.com/2020/06/connexion-a-une-base-de-donnees-postgresql-en-python.html

## create and insert tables

In [91]:
def create_tables(conn, table_req):
    try: 
        cursor = conn.cursor()
        cursor.execute(table_req)
        conn.commit()
        cursor.close()
    except (Exception, psycopg2.Error) as error :
        print ("Erreur lors de la création du table PostgreSQL", error)

In [92]:
def insert_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [93]:
conn = psycopg2.connect(
          user = "m139",
          password = "m139",
          host = "db-etu.univ-lyon2.fr",
          port = "5432",
          database = "m139"
    )

In [111]:
sql_create_hotel = '''CREATE TABLE HOTEL(
    id_hotel TEXT PRIMARY KEY NOT NULL,
    hotel VARCHAR(100)
  ); '''

create_tables(conn, sql_create_hotel)

sql_create_room = '''CREATE TABLE ROOM(
    id_room TEXT PRIMARY KEY NOT NULL,
    room_type TEXT,
    id_hotel TEXT,
    CONSTRAINT ROOM_ibfk_1 FOREIGN KEY (id_hotel) REFERENCES HOTEL(id_hotel)
  ); '''
create_tables(conn, sql_create_room)

sql_create_client = '''CREATE TABLE CLIENT(
    "id_client" TEXT PRIMARY KEY NOT NULL,
    country TEXT,
    nb_night INT,
    traveler_infos TEXT,
    review_title TEXT, 
    positive_review TEXT,
    negative_review TEXT, 
    usefulness_review TEXT
  ); '''
create_tables(conn, sql_create_client)

sql_create_date = '''CREATE TABLE DATE(
    id_date TEXT PRIMARY KEY NOT NULL,
    month_str TEXT,
    year INT,
    month_num INT,
    date DATE
  ); '''
create_tables(conn, sql_create_date)

sql_create_res = '''CREATE TABLE RESERVATION(
    id_reservation TEXT PRIMARY KEY NOT NULL,
    grade_review FLOAT,
    level_grade_review INT,
    id_room TEXT,
    id_date TEXT,
    id_client TEXT,
    CONSTRAINT RESERVATION_ibfk_1 FOREIGN KEY (id_room) REFERENCES ROOM(id_room),
    CONSTRAINT RESERVATION_ibfk_2 FOREIGN KEY (id_date) REFERENCES DATE(id_date),
    CONSTRAINT RESERVATION_ibfk_3 FOREIGN KEY (id_client) REFERENCES CLIENT(id_client)
  ); '''
create_tables(conn, sql_create_res)
    

Erreur lors de la création du table PostgreSQL relation "hotel" already exists

Erreur lors de la création du table PostgreSQL current transaction is aborted, commands ignored until end of transaction block

Erreur lors de la création du table PostgreSQL current transaction is aborted, commands ignored until end of transaction block

Erreur lors de la création du table PostgreSQL current transaction is aborted, commands ignored until end of transaction block

Erreur lors de la création du table PostgreSQL current transaction is aborted, commands ignored until end of transaction block



In [100]:
insert_values(conn, date, 'date')

the dataframe is inserted


In [101]:
insert_values(conn, hotel, 'hotel')

the dataframe is inserted


In [102]:
insert_values(conn, room, 'room')

the dataframe is inserted


In [103]:
insert_values(conn, client, 'client')

the dataframe is inserted


In [110]:
reservation = pd.read_csv("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables/reservation.csv",sep=';')
#le probleme l c'esr que id_room n'est pas toujours la, et du cop il rale .... 
#faudrait mettre un id_room pour les NA ??? 
#pcq il faudrait quand meme garder les res qui n'ont pas d'id_room... 

In [109]:
insert_values(conn, reservation, 'reservation')

Error: insert or update on table "reservation" violates foreign key constraint "reservation_ibfk_1"
DETAIL:  Key (id_room)=(NaN) is not present in table "room".



1

In [ ]:
DROP TABLE reservation;
DROP TABLE date;
DROP TABLE client;
DROP TABLE room;
DROP TABLE hotel;

In [ ]:
CREATE TABLE HOTEL(
    id_hotel TEXT PRIMARY KEY NOT NULL,
    hotel TEXT
  );
  
CREATE TABLE ROOM(
    id_room TEXT PRIMARY KEY NOT NULL,
    room_type TEXT,
    id_hotel TEXT,
    CONSTRAINT ROOM_ibfk_1 FOREIGN KEY (id_hotel) REFERENCES HOTEL(id_hotel)
  ); 
CREATE TABLE CLIENT(
    id_client TEXT PRIMARY KEY NOT NULL,
    country TEXT,
    nb_night INT,
    traveler_infos TEXT,
    review_title TEXT, 
    positive_review TEXT,
    negative_review TEXT, 
    usefulness_review TEXT
  );
  
CREATE TABLE DATE(
    id_date TEXT PRIMARY KEY NOT NULL,
    month_str TEXT,
    year INT,
    month_num INT,
    date DATE
  ); 
CREATE TABLE RESERVATION(
    id_reservation TEXT PRIMARY KEY NOT NULL,
    grade_review FLOAT,
    level_grade_review INT,
    id_room TEXT,
    id_date TEXT,
    id_client TEXT,
    CONSTRAINT RESERVATION_ibfk_1 FOREIGN KEY (id_room) REFERENCES ROOM(id_room),
    CONSTRAINT RESERVATION_ibfk_2 FOREIGN KEY (id_date) REFERENCES DATE(id_date),
    CONSTRAINT RESERVATION_ibfk_3 FOREIGN KEY (id_client) REFERENCES CLIENT(id_client)
  ); 

## get tables

In [ ]:
try:
    conn = psycopg2.connect(
          user = "m139",
          password = "m139",
          host = "db-etu.univ-lyon2.fr",
          port = "5432",
          database = "disney"
    )
    cur = conn.cursor()
    sql = "SELECT * FROM table"

    cur.execute(sql)
    res = cur.fetchall()

    cur.close()
    conn.close()
except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)


#recuperer les donnees de table
    col1 = []
    col2 = []
    col3 = []
    for i in range(0, len(res)):
        col1.append(res[i][0])
        col2.append(res[i][1])
        col3.append(res[i][2])
        
    #regrouper les donnees dans un dataframe pour la construction de la figure 
    table = pd.DataFrame({
        "col1" :  col2,  
        "col2" : col2,
        "col3": col3
    })